In [ ]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

#import julia; julia.install(quiet=True)
from julia import Main

import numpy     as np
import panel     as pn; pn.extension()
import holoviews as hv; hv.extension( "bokeh", logo=False)

from panel.interact import interact

def raster(img):  return hv.Image(img).opts(cmap="gray", xaxis=None, yaxis=None, frame_width=200, aspect='equal')

In [ ]:
%load_ext julia.magic

In [3]:
%%julia
using LinearAlgebra;

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Geodesics and Optimization of the Grassmannian</strong>
</div></div>

The Grassmannian has a natural notion of geodesics — shortest paths between subspaces.

In this notebook, we visualize these paths and explore how optimization algorithms move along them.


## Geodesics and Optimization on the Grassmannian

Subspaces live on the Grassmannian $\mathrm{Gr}(k, n)$ —  
a curved space with its own notion of distance and direction.

We explore how subspaces move along geodesics,  
and how optimization paths follow the geometry of this manifold.

---

### Geodesic Interpolation

Let $U, V \in \mathrm{Gr}(k, n)$ with orthonormal bases.  
Compute principal angles and vectors:

$$
U^\top V = A = L \Sigma R^\top
$$

Define geodesic path $U(t)$ from $U$ to $V$ as:

$$
U(t) = U L \cos(\Theta t) + Q \sin(\Theta t)
$$

where $\Theta = \operatorname{diag}(\theta_1, \dots, \theta_k)$  
and $Q$ is constructed to ensure orthonormality.

- $t = 0$ → $U$,  $t = 1$ → $V$  
- Intermediate $U(t)$ stays on the Grassmannian

---

### Visualization

- Animate smooth motion between subspaces  
- Show how angle evolution follows sinusoidal curves  
- Compare with linear interpolation (off-manifold)

---

### Optimization on $\mathrm{Gr}(k, n)$

Manifold-aware optimization respects curvature.  
Use the **geodesic gradient step** instead of Euclidean update.

Applications:

- Tracking PCA under drift  
- Gradient descent for subspace fitting  
- Subspace averaging

---

### Why Geodesics Matter

- Geodesics minimize subspace interpolation error  
- Optimization stays intrinsic to the space  
- Important in signal processing, machine learning, and low-rank models

---

### Next

We apply these tools to real tasks:  
drift detection, clustering, and geometry-aware learning.
